## Town04 Figure of 8 ring road study

In [49]:
# Boilerplate code, we assume that world with town 04 has been loaded in a parallel session
# boilerplate code
import carla 
import math 
import random 
import time 
import carla_helpers as helpers

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

## Place spectator for optimal viewing experience


In [50]:
import carla_helpers as helpers
#transform = helpers.get_spectator_transform(world)
transform = helpers.set_spectator_transform(world, (12.53, -430.17, 200.86), (-37.66, 57.56, 0))


=== Spectator Transform Set ===
Location: x=12.53, y=-430.17, z=200.86
Rotation: pitch=-37.66, yaw=57.56, roll=0.00


## Lane plotting

In [58]:
# helper functions
def find_eight_lane_roads(world):
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    
    road_lanes = {}
    for wp in waypoints:
        road_id = wp.road_id
        if road_id not in road_lanes:
            road_lanes[road_id] = set()
        road_lanes[road_id].add(wp.lane_id)
    
    eight_lane_roads = [road_id for road_id, lanes in road_lanes.items() 
                       if len(lanes) == 8]
    
    if eight_lane_roads:
        print(f"Roads with 8 lanes:")
        for road_id in eight_lane_roads:
            print(f"Road ID: {road_id}, Lanes: {sorted(list(road_lanes[road_id]))}")
    else:
        print("No roads with exactly 8 lanes found")
    
    return eight_lane_roads

# Usage:
eight_lane_roads = find_eight_lane_roads(world)
# eight_lane_roads

def place_markers_on_lane(world, eight_lane_roads, target_lane_id=-4):
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    
    # Filter waypoints for target roads and lane
    target_waypoints = [wp for wp in waypoints 
                       if wp.road_id in eight_lane_roads 
                       and wp.lane_id == target_lane_id]
    
    # Place markers
    markers = []
    marker_color = carla.Color(r=255, g=0, b=0)  # Red markers
    marker_life_time = 20.0  # Markers will persist for 20 seconds
    
    for wp in target_waypoints:
        location = wp.transform.location
        # Draw a sphere at each waypoint
        marker = world.debug.draw_point(
            location,
            size=0.1,  # 10cm sphere
            color=marker_color,
            life_time=marker_life_time
        )
        markers.append(marker)
    
    print(f"Placed {len(markers)} markers on lane {target_lane_id} "
          f"across {len(eight_lane_roads)} roads")
    
    return markers, target_waypoints

# Usage:
markers, target_waypoints = place_markers_on_lane(world, eight_lane_roads)


Roads with 8 lanes:
Road ID: 6, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 35, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 36, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 37, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 38, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 39, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 40, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 41, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 42, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 43, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 45, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 46, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 47, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 48, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 49, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 50, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Placed 1246 markers on lane -4 across 16 roads


## Place markers on one road segment (road_id) only

In [14]:
#markers = place_markers_on_lane(world, [6])
markers = place_markers_on_lane(world, [50])

Placed 89 markers on lane -4 across 1 roads


In [33]:
type(markers) # tuple
type(markers[0]) # list
type(markers[1]) # list
markers[1]
type(markers[1][0]) # carla.Waypoint

carla.libcarla.Waypoint

## Determine if markers are geo-sequencial

In [37]:
def analyze_marker_sequence(markers, distance_threshold=2.5):
    if len(markers) < 2:
        print("At least two markers are required for analysis")
        return True
        
    sequence_breaks = []
    for i in range(len(markers) - 1):
        loc1 = markers[1][i].transform.location
        loc2 = markers[1][i + 1].transform.location
        
        distance = ((loc2.x - loc1.x)**2 + 
                   (loc2.y - loc1.y)**2 + 
                   (loc2.z - loc1.z)**2)**0.5
                   
        if distance > distance_threshold:
            sequence_breaks.append((i, distance))
    
    if sequence_breaks:
        print(f"Found {len(sequence_breaks)} breaks in sequence:")
        for break_idx, dist in sequence_breaks:
            print(f"Break between markers {break_idx} and {break_idx+1}, distance: {dist:.2f}m")
        return False
        
    return True

analyze_marker_sequence(markers)

True

## Determine if waypoints are sequencial

In [25]:
type(target_waypoints) # list
type(target_waypoints[0]) # carla.libcarla.Waypoint
for method in dir(target_waypoints[0]):
    print(method)
type(target_waypoints[0].transform) # carla.libcarla.Transform 
for method in dir(target_waypoints[0].transform):
    print(method)
print(type(target_waypoints[0].transform.location)) # <class 'carla.libcarla.Location'>
for method in dir(target_waypoints[0].transform.location):
    print(method)
type(target_waypoints[0].transform.location.x) # float

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
get_junction
get_landmarks
get_landmarks_of_type
get_left_lane
get_right_lane
id
is_intersection
is_junction
junction_id
lane_change
lane_id
lane_type
lane_width
left_lane_marking
next
next_until_lane_end
previous
previous_until_lane_start
right_lane_marking
road_id
s
section_id
transform
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__instance_size__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
get_forward_vector
get_inverse_matrix
get_matrix
get_right_vector
get_up_vector
location
rotation
transform
transform_vector
<class 'carla.libcarla.Location'>
_

float

In [40]:
def analyze_road_sequence(world, road_id, distance_threshold=2.5, debug=False):
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    
    if debug:
        print(f"Analyzing road_id: {road_id}")
    
    # Get all waypoints for this road, organized by lane
    road_lanes = {}
    for wp in waypoints:
        if wp.road_id == road_id:
            if wp.lane_id not in road_lanes:
                road_lanes[wp.lane_id] = []
            road_lanes[wp.lane_id].append(wp)
    
    # Sort waypoints in each lane by s-value
    for lane_id in road_lanes:
        if debug:
            print(f"Checking sequence for lane_id: {lane_id}")
        road_lanes[lane_id].sort(key=lambda x: x.s)
    
    # Check sequence for each lane
    sequence_breaks = {}
    for lane_id, lane_waypoints in road_lanes.items():
        if len(lane_waypoints) < 2:
            continue
        breaks = []
        for i in range(len(lane_waypoints) - 1):
            loc1 = lane_waypoints[i].transform.location
            loc2 = lane_waypoints[i + 1].transform.location
            distance = ((loc2.x - loc1.x)**2 + 
                       (loc2.y - loc1.y)**2 + 
                       (loc2.z - loc1.z)**2)**0.5
            if distance > distance_threshold:
                breaks.append((i, distance))
        if breaks:
            sequence_breaks[lane_id] = breaks
    
    # Report findings
    if sequence_breaks:
        print(f"Found sequence breaks in road {road_id}:")
        for lane_id, breaks in sequence_breaks.items():
            print(f"\nLane {lane_id}:")
            for break_idx, dist in breaks:
                print(f"Break between waypoints {break_idx} and {break_idx+1}, distance: {dist:.2f}m")
        return False
    
    print(f"Road {road_id} has continuous waypoint sequences in all lanes")
    return True
road_id = 6
analyze_road_sequence(world, road_id, distance_threshold=2.5, debug=True)

Analyzing road_id: 6
Checking sequence for lane_id: -4
Checking sequence for lane_id: -3
Checking sequence for lane_id: -2
Checking sequence for lane_id: -1
Checking sequence for lane_id: 3
Checking sequence for lane_id: 4
Checking sequence for lane_id: 5
Checking sequence for lane_id: 6
Road 6 has continuous waypoint sequences in all lanes


True

In [52]:
carla_map = world.get_map()
waypoints = carla_map.generate_waypoints(2.0)
for method in dir(waypoints[0]):
    print(method)
type(waypoints)    

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
get_junction
get_landmarks
get_landmarks_of_type
get_left_lane
get_right_lane
id
is_intersection
is_junction
junction_id
lane_change
lane_id
lane_type
lane_width
left_lane_marking
next
next_until_lane_end
previous
previous_until_lane_start
right_lane_marking
road_id
s
section_id
transform


list

In [69]:
waypoints[0].right_lane_marking.type

carla.libcarla.LaneMarkingType.Solid

## check all road_ids for non-sequential waypoint

In [48]:
def get_unique_road_ids(waypoints):
    return sorted(list(set(wp.road_id for wp in waypoints)))

road_ids = get_unique_road_ids(waypoints)

for road_id in road_ids:
    analyze_road_sequence(world, road_id, distance_threshold=2.5)



Road 0 has continuous waypoint sequences in all lanes
Road 1 has continuous waypoint sequences in all lanes
Road 2 has continuous waypoint sequences in all lanes
Road 3 has continuous waypoint sequences in all lanes
Road 4 has continuous waypoint sequences in all lanes
Road 5 has continuous waypoint sequences in all lanes
Road 6 has continuous waypoint sequences in all lanes
Road 7 has continuous waypoint sequences in all lanes
Road 8 has continuous waypoint sequences in all lanes
Road 9 has continuous waypoint sequences in all lanes
Road 10 has continuous waypoint sequences in all lanes
Road 11 has continuous waypoint sequences in all lanes
Road 12 has continuous waypoint sequences in all lanes
Road 13 has continuous waypoint sequences in all lanes
Road 14 has continuous waypoint sequences in all lanes
Road 15 has continuous waypoint sequences in all lanes
Road 16 has continuous waypoint sequences in all lanes
Road 17 has continuous waypoint sequences in all lanes
Road 18 has continuo

## List road lanes

In [54]:
def get_unique_lane_ids(waypoints, road_id):
    return sorted(list(set(wp.lane_id for wp in waypoints if wp.road_id == road_id)))

road_id = 6
lane_ids = get_unique_lane_ids(waypoints, road_id)  
print(lane_ids) # [-4, -3, -2, -1, 3, 4, 5, 6]

road_id = 38
lane_ids = get_unique_lane_ids(waypoints, road_id)  
lane_ids



[-4, -3, -2, -1, 3, 4, 5, 6]


[-4, -3, -2, -1, 3, 4, 5, 6]

In [61]:
eight_lane_roads

[6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]

# Plot inner late of figure of 8 ring road, town 04

In [65]:
road_ids = [6, 35]
markers = place_markers_on_lane(world, road_ids)

# for road_id in eight_lane_roads:
#     place_markers_on_lane(world, [road_id])

Placed 175 markers on lane -4 across 2 roads


In [64]:
## Waypoints set at distance > 2.5m
for waypoint in waypoints:
    if waypoint.road_id == 6:
        print(waypoint.id)
        # print(waypoint.s)

4641250760981045379
4592942668712833820
11983327098621662466
5224660670092596996
6982654154847670249
6129230666532924672
5034693531769484326
2827203210415223208
18313294055911148380
15644884804647786691
3383236460981022085
7460701396986866395
8423436749056312822
9516529405160445831
7374316201285725625
3090671597044257503
9387395423480887448
16864857345223855
11347369723081038233
3248915232482796439
10806400343159910738
11881993350336530155
4654049739649610341
17483696824142191507
2811769558798115716
13458593866777907099
12933423164516232109
10559221071065543683
11217352454628465086
7135836977279816703
8990204711485311697
4229121148518471847
5644134607749557701
6752368382134331394
12251833964545368092
356858600335447322
16762126860177900383
10096639739408483614
8779535702564972536
6065289533055955062
604644998855766037
5973068758167605170
4105289373958060140
14865034262452006250
3407846882579146319
14004514317817939822
4103668378455050984
3492852901020968294
10576358984672155825
3957045

## world object methods

In [70]:
for method in dir(world):
    print(method)

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
apply_color_texture_to_object
apply_color_texture_to_objects
apply_float_color_texture_to_object
apply_float_color_texture_to_objects
apply_settings
apply_textures_to_object
apply_textures_to_objects
cast_ray
debug
enable_environment_objects
freeze_all_traffic_lights
get_actor
get_actors
get_blueprint_library
get_environment_objects
get_level_bbs
get_lightmanager
get_map
get_names_of_all_objects
get_random_location_from_navigation
get_settings
get_snapshot
get_spectator
get_traffic_light
get_traffic_light_from_opendrive_id
get_traffic_lights_from_waypoint
get_traffic_lights_in_junction
get_traffic_sign
get_vehicles_light_states
get_weather
ground_projection
id
load_map_layer
on_tick
project_point
remove_on_tick
reset_all_tra

## Figure of 8 waypoings

In [71]:
def validate_figure8_waypoints(world):
    """Get and validate waypoints for figure-8 track."""
    road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    target_lane_id = -4
    waypoint_spacing = 2.0
    
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(waypoint_spacing)
    
    # Filter and sort waypoints
    road_waypoints = {}
    for road_id in road_sequence:
        road_waypoints[road_id] = [wp for wp in waypoints 
                                 if wp.road_id == road_id and wp.lane_id == target_lane_id]
        road_waypoints[road_id].sort(key=lambda x: x.s)
        
        print(f"Road {road_id}: {len(road_waypoints[road_id])} waypoints")
        
        # Validate waypoint spacing
        if len(road_waypoints[road_id]) > 1:
            first = road_waypoints[road_id][0].transform.location
            last = road_waypoints[road_id][-1].transform.location
            distance = ((last.x - first.x)**2 + 
                       (last.y - first.y)**2 + 
                       (last.z - first.z)**2)**0.5
            print(f"Road length: {distance:.1f}m")
    
    return road_waypoints

road_waypoints = validate_figure8_waypoints(world)
road_waypoints

Road 6: 32 waypoints
Road length: 62.1m
Road 35: 143 waypoints
Road length: 225.2m
Road 36: 41 waypoints
Road length: 80.0m
Road 37: 21 waypoints
Road length: 36.6m
Road 38: 123 waypoints
Road length: 233.5m
Road 39: 67 waypoints
Road length: 132.0m
Road 40: 119 waypoints
Road length: 236.3m
Road 41: 105 waypoints
Road length: 198.2m
Road 42: 17 waypoints
Road length: 32.0m
Road 43: 17 waypoints
Road length: 32.2m
Road 45: 293 waypoints
Road length: 499.0m
Road 46: 70 waypoints
Road length: 142.1m
Road 47: 57 waypoints
Road length: 112.0m
Road 48: 32 waypoints
Road length: 62.0m
Road 49: 20 waypoints
Road length: 37.8m
Road 50: 89 waypoints
Road length: 150.8m


{6: [<carla.libcarla.Waypoint at 0x7f574718e5b0>,
 35: [<carla.libcarla.Waypoint at 0x7f57471b52f0>,
 36: [<carla.libcarla.Waypoint at 0x7f5747157ad8>,
 37: [<carla.libcarla.Waypoint at 0x7f5747161030>,
 38: [<carla.libcarla.Waypoint at 0x7f574716a298>,
 39: [<carla.libcarla.Waypoint at 0x7f5747194558>,
 40: [<carla.libcarla.Waypoint at 0x7f5747151870>,
 41: [<carla.libcarla.Waypoint at 0x7f5747099a28>,
 42: [<carla.libcarla.Waypoint at 0x7f57500642f0>,
 43: [<carla.libcarla.Waypoint at 0x7f575005de48>,
 45: [<carla.libcarla.Waypoint at 0x7f5747274298>,
 46: [<carla.libcarla.Waypoint at 0x7f57470faef8>,
 47: [<carla.libcarla.Waypoint at 0x7f57501137c0>,
 48: [<carla.libcarla.Waypoint at 0x7f57500fe608>,
 49: [<carla.libcarla.Waypoint at 0x7f5750110298>,
 50: [<carla.libcarla.Waypoint at 0x7f5750108768>,
  <carla.libcarla.Waypoint at 0x7f574710c030>]}

## road waypoints

In [78]:
def drive_figure_eight(world, vehicle):
    """Drive vehicle around Town04's figure-eight."""
    road_waypoints = validate_figure8_waypoints(world)
    road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    target_speed = 30  # km/h
    
    try:
        while True:
            for road_id in road_sequence:
                for wp in road_waypoints[road_id]:
                    control = compute_control(vehicle, wp, target_speed)
                    vehicle.apply_control(control)
                    
                    if world.get_settings().synchronous_mode:
                        world.tick()
                    else:
                        time.sleep(0.1)  # Adjust timing as needed
                        
    except KeyboardInterrupt:
        control = carla.VehicleControl(throttle=0, brake=1)
        vehicle.apply_control(control)

def compute_control(vehicle, target_wp, target_speed):
    """Compute vehicle control to reach waypoint."""
    control = carla.VehicleControl()
    
    # Get current state
    current_transform = vehicle.get_transform()
    current_velocity = vehicle.get_velocity()
    speed = 3.6 * math.sqrt(current_velocity.x**2 + current_velocity.y**2)  # km/h
    
    # Steering based on angle difference
    forward = current_transform.get_forward_vector()
    right = current_transform.get_right_vector()
    up = current_transform.get_up_vector()
    target_vector = target_wp.transform.location - current_transform.location
    
    forward_dot = (forward.x * target_vector.x + forward.y * target_vector.y) 
    right_dot = (right.x * target_vector.x + right.y * target_vector.y)
    
    steering = math.atan2(right_dot, forward_dot) / math.pi
    control.steer = max(-1.0, min(1.0, steering))
    
    # Speed control
    if speed < target_speed:
        control.throttle = 0.7
        control.brake = 0.0
    else:
        control.throttle = 0.0
        control.brake = 0.3
        
    return control

dict